<a href="https://colab.research.google.com/github/mohamedsayed18/Pose_estimation/blob/master/TFlite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import tensorflow as tf
import cv2 
from matplotlib import pyplot as plt

In [0]:
#load my drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Load the image
img = cv2.imread("/content/drive/My Drive/innopolis/1st year courses/2nd term/CV/GP/data/im0002.jpg")
new_im = cv2.resize(img, (257,257))

In [0]:
# Load TFLite model and allocate tensors.
path = "/content/drive/My Drive/innopolis/1st year courses/2nd term/CV/GP/posenet.tflite"
interpreter = tf.lite.Interpreter(model_path=path)
interpreter.allocate_tensors()

In [0]:
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [0]:
#img = cv2.imread("/content/drive/My Drive/innopolis/1st year courses/2nd term/CV/GP/data/im0002.jpg")
# Test model on random input data.
input_shape = input_details[0]['shape']   # get the shape
input_index = interpreter.get_input_details()[0]['index']   # get the index
new_im = new_im.reshape((input_shape))  #reshape
x = np.asarray(new_im).astype(np.float32)   # convert to float32


interpreter.set_tensor(input_index, x)
interpreter.invoke()

In [0]:
# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data.shape)
print (output_data)

(1, 9, 9, 17)
[[[[-12.103102  -13.412443  -11.969072  ... -10.503205   -9.950239
     -9.60874  ]
   [-10.471811  -12.704063  -11.942806  ... -10.832616   -9.742912
     -9.1458   ]
   [-12.99206   -13.86536   -12.508657  ... -11.341109  -10.686723
    -10.263073 ]
   ...
   [-16.958427  -17.114805  -17.100107  ... -14.025278  -13.521445
    -13.995409 ]
   [-13.849909  -15.714219  -15.785658  ... -12.887864  -11.957799
    -12.980194 ]
   [-10.165682  -11.682789  -11.686472  ...  -9.528605  -10.3153925
    -10.504871 ]]

  [[-10.031876  -12.556335  -11.943453  ... -12.512616  -11.236356
    -10.890425 ]
   [ -9.267262  -10.342054  -10.820351  ... -10.049693  -10.424857
     -9.2112465]
   [-12.234393  -11.214573  -10.442067  ...  -9.637682  -10.024852
     -9.807783 ]
   ...
   [-14.327654  -13.490206  -13.933462  ... -11.042529  -10.485789
    -11.889557 ]
   [-14.377781  -16.434563  -15.808243  ... -12.952103  -13.001435
    -13.376442 ]
   [-10.034048  -10.46348   -11.186944  ... -